# Analyzing River Thames Water Levels
Time series data is everywhere, from watching your stock portfolio to monitoring climate change, and even live-tracking as local cases of a virus become a global pandemic. In this project, you’ll work with a time series that tracks the tide levels of the Thames River. You’ll first load the data and inspect it data visually, and then perform calculations on the dataset to generate some summary statistics. You’ll end by reducing the time series to its component attributes and analyzing them. 

The original dataset is available from the British Oceanographic Data Center.

Here's a map of the locations of the tidal meters along the River Thames in London.

![](locations.png)

The provided datasets are in the `data` folder in this workspace. For this project, you will work with one of these files, `10-11_London_Bridge.txt`, which contains comma separated values for water levels in the Thames River at the London Bridge. After you've finished the project, you can use your same code to analyze data from the other files (at other spots in the UK where tidal data is collected) if you'd like. 

The TXT file contains data for three variables, described in the table below. 

| Variable Name | Description | Format |
| ------------- | ----------- | ------ |
| Date and time | Date and time of measurement to GMT. Note the tide gauge is accurate to one minute. | dd/mm/yyyy hh:mm:ss |
| Water level | High or low water level measured by tide meter. Tide gauges are accurate to 1 centimetre. | metres (Admiralty Chart Datum (CD), Ordnance Datum Newlyn (ODN or Trinity High Water (THW)) | 
| Flag | High water flag = 1, low water flag = 0 | Categorical (0 or 1) |



In [187]:
# We've imported your first Python package for you, along with a function you will need called IQR
import pandas as pd  
import numpy as np

def IQR(column): 
    """ Calculates the interquartile range (IQR) for a given DataFrame column using the quantile method """
    q25, q75 = column.quantile([0.25, 0.75])
    return q75-q25

# Begin coding here ...

## Getting Started
I first imported the data into a DataFrame I titled `lb_water_levels`. I then printed out the info and the first few rows of the DataFrame to get a better idea of what I was working with. Next, I cleaned up the DataFrame to make it easier for me to work with. 
I started by removing any whitespace in the column names. I then converted the data in the 'Date and time' column to the dtype `datetime`, removed the column 'HW=1 or LW=0', converted the data in the 'flag' column to the dtype `category`, converted the data in the 'water levels (m ODN)' column to the dtype `float`, and renamed the 'water levels (m ODN)' column to 'water levels'. 

In [188]:
lb_water_levels = pd.read_csv('data/10-11_London_Bridge.txt')
print(lb_water_levels.info())
lb_water_levels.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115503 entries, 0 to 115502
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Date and time         115503 non-null  object 
 1    water level (m ODN)  115503 non-null  object 
 2    flag                 115503 non-null  int64  
 3    HW=1 or LW=0         0 non-null       float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.5+ MB
None


,Date and time,water level (m ODN),flag,HW=1 or LW=0
0,01/05/1911 15:40:00,3.7130,1,NaN
1,02/05/1911 11:25:00,-2.9415,0,NaN
2,02/05/1911 16:05:00,3.3828,1,NaN
3,03/05/1911 11:50:00,-2.6367,0,NaN
4,03/05/1911 16:55:00,2.9256,1,NaN


In [189]:
lb_water_levels.columns = lb_water_levels.columns.str.strip()
lb_water_levels['Date and time'] = pd.to_datetime(lb_water_levels['Date and time'], errors='coerce', dayfirst=True)
lb_water_levels.drop(columns=['HW=1 or LW=0'], inplace=True)
lb_water_levels['flag'] = lb_water_levels['flag'].astype('category')
lb_water_levels['water level (m ODN)'] = lb_water_levels['water level (m ODN)'].astype('float')
lb_water_levels.rename(columns={'water level (m ODN)': 'water_level'}, inplace = True)
print(lb_water_levels.info())
lb_water_levels.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115503 entries, 0 to 115502
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date and time  115503 non-null  datetime64[ns]
 1   water_level    115489 non-null  float64       
 2   flag           115503 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 1.9 MB
None


,Date and time,water_level,flag
0,1911-05-01 15:40:00,3.7130,1
1,1911-05-02 11:25:00,-2.9415,0
2,1911-05-02 16:05:00,3.3828,1
3,1911-05-03 11:50:00,-2.6367,0
4,1911-05-03 16:55:00,2.9256,1


## Question 1: Find the mean, median and interquartile range for high- and low-tide data and save them as two separate `pandas` Series. 

In [190]:
low_tide_stats = lb_water_levels[lb_water_levels['flag'] == 0]\
                    ['water_level'].agg([np.mean, np.median, IQR])
high_tide_stats = lb_water_levels[lb_water_levels['flag'] == 1]\
                    ['water_level'].agg([np.mean, np.median, IQR])
print(low_tide_stats)
print(high_tide_stats)

mean     -2.383737
median   -2.412900
IQR       0.538200
Name: water_level, dtype: float64
mean      3.318373
median    3.352600
IQR       0.743600
Name: water_level, dtype: float64


## Question 2: Calculate the annual percentage of days with very high tide levels (90th percentile of high tide days) and low-tide days (below the 10th percentile), and store the results for each year as floats in two two-column DataFrames with the index reset.
I started by creating a column that only contained the year of each observation. To find the annual percentage of days with very high tide levels, I first created a new DataFrame that contained only high tide observations. I then calculated the 90th percentile of the water level of high tide days. I finished by calculating the total number of days each year that had tide levels above the 90th percentile, dividing by the total number of days that year that observed high tide, and saving the results as a DataFrame. I repeated this process to calculate the low tide ratio using the 10th percentile. 

In [191]:
#Creating new 'year' column
import datetime as dt
lb_water_levels['year'] = lb_water_levels['Date and time'].dt.year
lb_water_levels.head()

,Date and time,water_level,flag,year
0,1911-05-01 15:40:00,3.7130,1,1911
1,1911-05-02 11:25:00,-2.9415,0,1911
2,1911-05-02 16:05:00,3.3828,1,1911
3,1911-05-03 11:50:00,-2.6367,0,1911
4,1911-05-03 16:55:00,2.9256,1,1911


In [192]:
#Creating a new DataFrame that only contains high tide level data
lb_water_levels_high = lb_water_levels[lb_water_levels['flag'] == 1]

#Finding the 90th percentile of the high tide data
v_high_tide = lb_water_levels_high['water_level'].quantile(0.90)

#Finding the number of days each year that had high tides above the 90th percentile
num_v_high_days = lb_water_levels_high[lb_water_levels_high['water_level'] > v_high_tide].groupby('year')['water_level'].agg(len)

#Finding the number of days each year that high tide data was recorded
num_high_days_total = lb_water_levels_high.groupby('year')['water_level'].agg(len)

#Calculating the % of days with very high tides each year and saving as a DataFrame
high_tide_ratio = pd.DataFrame(num_v_high_days/num_high_days_total)
high_tide_ratio.reset_index(inplace=True)
print(high_tide_ratio)
print(high_tide_ratio.info())

    year  water_level
0   1911     0.004098
1   1912     0.032316
2   1913     0.082212
3   1914     0.055233
4   1915     0.045045
..   ...          ...
80  1991     0.096317
81  1992     0.103253
82  1993     0.145923
83  1994     0.150355
84  1995     0.170213

[85 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         85 non-null     int64  
 1   water_level  85 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 1.5 KB
None


In [193]:
#Creating a new DataFrame that only contains low tide data
lb_water_levels_low = lb_water_levels[lb_water_levels['flag'] == 0]

#Finding the 10th quantile of the low tide data
v_low_tide = lb_water_levels_low['water_level'].quantile(0.10)

#Calculating number of days each year that have low tides below 10th quantile
num_v_low_days = lb_water_levels_low[lb_water_levels_low['water_level'] < v_low_tide].groupby('year')['water_level'].agg(len)

#Calculating number of days each year that listed low tide data
num_low_days_total = lb_water_levels_low.groupby('year')['water_level'].agg(len)

#Calculating the % of days with very low tides each year and saving as a DataFrame
low_tide_ratio = pd.DataFrame(num_v_low_days/num_low_days_total)
low_tide_ratio.reset_index(inplace=True)
print(low_tide_ratio)
print(low_tide_ratio.info())

    year  water_level
0   1911     0.060606
1   1912     0.066667
2   1913     0.022388
3   1914     0.039017
4   1915     0.033435
..   ...          ...
80  1991     0.150355
81  1992     0.107496
82  1993     0.112696
83  1994     0.106383
84  1995     0.107801

[85 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         85 non-null     int64  
 1   water_level  85 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 1.5 KB
None


## Question 3: Create a dictionary named solution with a summary of your data analysis, with these key-value pairs:

`{high_statistics: high-tide stats, low_statistics: low-tide stats, very_high_ratio: high-tide ratio data, very_low_ratio: low-tide ratio data}`

In [194]:
solution = {'high_statistics': high_tide_stats, 'low_statistics': low_tide_stats,\
            'very_high_ratio': high_tide_ratio,'very_low_ratio': low_tide_ratio}
print(solution)

{'high_statistics': mean      3.318373
median    3.352600
IQR       0.743600
Name: water_level, dtype: float64, 'low_statistics': mean     -2.383737
median   -2.412900
IQR       0.538200
Name: water_level, dtype: float64, 'very_high_ratio':     year  water_level
0   1911     0.004098
1   1912     0.032316
2   1913     0.082212
3   1914     0.055233
4   1915     0.045045
..   ...          ...
80  1991     0.096317
81  1992     0.103253
82  1993     0.145923
83  1994     0.150355
84  1995     0.170213

[85 rows x 2 columns], 'very_low_ratio':     year  water_level
0   1911     0.060606
1   1912     0.066667
2   1913     0.022388
3   1914     0.039017
4   1915     0.033435
..   ...          ...
80  1991     0.150355
81  1992     0.107496
82  1993     0.112696
83  1994     0.106383
84  1995     0.107801

[85 rows x 2 columns]}
